In [1]:
import pandas as pd
import numpy as np
from geopy.distance import distance
import pydeck as pdk
import pickle
import requests
import json

In [2]:
model = pickle.load(open('../../../capstone_other/files_too_large_for_github/random_forest.pickle', 'rb'))

In [3]:
data = pd.read_csv('../../../capstone_other/files_too_large_for_github/ML_table_with_availability_clusters.csv')

In [4]:
new = data.groupby('dock_id').apply(pd.DataFrame.sample, random_state = 5, n=1).reset_index(drop=True)

In [5]:
new1 = new[['dock_id', 'latitude', 'longitude', 'tot_docks']]

In [6]:
new2 = new[['weekday_cluster', 'weekend_cluster']]

In [7]:
request = requests.get('https://api.openweathermap.org/data/2.5/onecall?lat=40.7812&lon=-73.9665&exclude=minutely,current,daily,alerts&units=metric&appid=404310456b8e1c31228341dd6c95dd04')
response = request.json()

In [8]:
weather_df = pd.DataFrame.from_dict(response['hourly'])[['dt', 'temp','pop']]
weather_df['dt'] = pd.to_datetime(weather_df['dt'], unit = 's')

In [9]:
weather_df['pop'] = weather_df['pop'].apply(lambda x: round(x))

In [10]:
day_of_week_conversion = {0:2, 1:3, 2:4, 3:5, 4:6, 5:7, 6:1}

In [11]:
full = pd.DataFrame()
for row in list(weather_df.iterrows()):
    to_append = pd.DataFrame()
    date = row[1]['dt']
    month = date.month
    to_append['month'] = np.ones(len(new), dtype=int) * month
    day_of_week = date.dayofweek
    sql_day_of_week = day_of_week_conversion[day_of_week]
    to_append['num_day'] = np.ones(len(new), dtype=int) * sql_day_of_week
    hour = date.hour
    to_append['hour'] = np.ones(len(new), dtype=int) * hour
    to_append = pd.concat([to_append, new1], axis = 1)
    temp = row[1]['temp']
    rain = row[1]['pop']
    to_append['tmp'] = np.ones(len(new)) * temp
    to_append['rain'] = np.ones(len(new), dtype=int) * rain
    to_append['datetime'] = [date for _ in range(len(new))]
    to_append = pd.concat([to_append, new2], axis = 1)
    full = pd.concat([full, to_append], axis = 0)

In [12]:
dock_id_series = full['dock_id']
datetime_series = full['datetime']

In [13]:
full.drop(['dock_id', 'datetime'], axis = 1, inplace = True)

In [14]:
predictions = model.best_estimator_.predict(full)

In [15]:
full['predictions'] = predictions
full['dock_id'] = dock_id_series
full['datetime'] = datetime_series

In [16]:
full['avail_bikes_proportion'] = full['predictions']/full['tot_docks']

In [17]:
user_input = pd.to_datetime('2021-12-19 14:00:00')

In [18]:
month = user_input.month
num_day = day_of_week_conversion[user_input.dayofweek]
hour = user_input.hour

In [51]:
full.to_csv('../dash_app/data/robert/dataframe_for_live_predictions.csv')

# Load .csv file for live predictions

In [52]:
full = pd.read_csv('../dash_app/data/robert/dataframe_for_live_predictions.csv')

In [19]:
query = full[(full['month'] == month) & (full['num_day'] == num_day) & (full['hour'] == hour)]

In [20]:
def manhattan_distance(start_lat, start_lon, end_lat, end_lon):
    dist = distance((start_lat, start_lon), (start_lat, end_lon)).miles + \
           distance((end_lat, end_lon), (start_lat, end_lon)).miles
    return dist

In [21]:
low_bike_threshold = 1/3
high_bike_threshold = 2/3
max_distance = 2
max_bikes_rebalanced = 500
min_cargo_size = 3

In [22]:
data_low = query[query['avail_bikes_proportion'] <= low_bike_threshold]

In [23]:
data_high = query[query['avail_bikes_proportion'] >= high_bike_threshold]

In [24]:
data_low['deficit'] = round((low_bike_threshold - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')


<ipython-input-24-d746bb208678>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_low['deficit'] = round((low_bike_threshold - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')


In [25]:
data_high['surplus'] = round((data_high['avail_bikes_proportion'] - high_bike_threshold) * data_high['tot_docks']).astype('int')


<ipython-input-25-d9c66eeb9a9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_high['surplus'] = round((data_high['avail_bikes_proportion'] - high_bike_threshold) * data_high['tot_docks']).astype('int')


In [26]:
data_low = data_low.sort_values(by = 'deficit', ascending = False)

In [27]:
data_high = data_high.sort_values(by = 'surplus', ascending = False)

In [28]:
rebalancing_dict = {}

In [29]:
low_copy = data_low.copy()
high_copy = data_high.copy()

In [30]:
bikes_rebalanced = 0
for low in low_copy.index:
    if low_copy.loc[low, 'deficit'] == 0:
        continue
    for high in high_copy.index:
        if high_copy.loc[high, 'surplus'] == 0:
            continue
        if manhattan_distance(low_copy.loc[low, 'latitude'], low_copy.loc[low, 'longitude'], 
                              high_copy.loc[high, 'latitude'], high_copy.loc[high, 'longitude']) < max_distance:
            stations_key = (low_copy.loc[low, 'dock_id'], high_copy.loc[high, 'dock_id'])
            change = min(low_copy.loc[low, 'deficit'], high_copy.loc[high, 'surplus'])
            low_copy.loc[low, 'deficit'] -= change
            high_copy.loc[high, 'surplus'] -= change
            bikes_rebalanced += change
            if stations_key in rebalancing_dict.keys():
                rebalancing_dict[stations_key] += change
            else:
                rebalancing_dict[stations_key] = change
            if low_copy.loc[low, 'deficit'] == 0:
                break

In [31]:
rebalancing_dict

{(469, 3161): 4,
 (469, 3167): 3,
 (469, 3158): 1,
 (469, 3137): 1,
 (167, 168): 9,
 (167, 462): 3,
 (164, 3128): 1,
 (3146, 3379): 9,
 (3146, 3318): 3,
 (3141, 3318): 4,
 (3141, 3312): 3,
 (3141, 3296): 1,
 (3437, 3502): 5,
 (3437, 3509): 2,
 (3437, 3387): 2,
 (3295, 3500): 5,
 (3295, 3338): 4,
 (3295, 3497): 1,
 (3295, 3490): 1,
 (3462, 334): 4,
 (3462, 3463): 1,
 (3452, 539): 8,
 (3452, 2002): 2,
 (3127, 3112): 3,
 (3127, 3109): 3,
 (3127, 3123): 2,
 (2012, 3472): 8,
 (195, 3427): 7,
 (3479, 3409): 7,
 (3418, 3429): 6,
 (3310, 3429): 5,
 (3310, 3477): 1,
 (524, 462): 6,
 (3306, 3335): 6,
 (3102, 2002): 5,
 (448, 3472): 3,
 (448, 3224): 2,
 (3358, 3335): 2,
 (3358, 3419): 2,
 (3456, 3093): 4,
 (3354, 3419): 4,
 (3121, 3105): 1,
 (174, 3427): 4,
 (3050, 3046): 3,
 (3050, 3070): 1,
 (3449, 3093): 1,
 (3449, 3090): 2,
 (316, 355): 3,
 (406, 355): 3,
 (365, 3419): 2,
 (365, 3409): 1,
 (3064, 343): 2,
 (3064, 3070): 1,
 (3077, 3090): 2,
 (3077, 3092): 1,
 (3094, 3067): 1,
 (491, 3467): 3,

In [32]:
sorted_rebalancing = dict(sorted(rebalancing_dict.items(), key=lambda x: x[1], reverse = True))

In [33]:
filtered_rebalancing = {key: value for key, value in sorted_rebalancing.items() if value >= min_cargo_size}

In [34]:
filtered_rebalancing

{(167, 168): 9,
 (3146, 3379): 9,
 (3452, 539): 8,
 (2012, 3472): 8,
 (195, 3427): 7,
 (3479, 3409): 7,
 (3418, 3429): 6,
 (524, 462): 6,
 (3306, 3335): 6,
 (3437, 3502): 5,
 (3295, 3500): 5,
 (3310, 3429): 5,
 (3102, 2002): 5,
 (469, 3161): 4,
 (3141, 3318): 4,
 (3295, 3338): 4,
 (3462, 334): 4,
 (3456, 3093): 4,
 (3354, 3419): 4,
 (174, 3427): 4,
 (469, 3167): 3,
 (167, 462): 3,
 (3146, 3318): 3,
 (3141, 3312): 3,
 (3127, 3112): 3,
 (3127, 3109): 3,
 (448, 3472): 3,
 (3050, 3046): 3,
 (316, 355): 3,
 (406, 355): 3,
 (491, 3467): 3}

In [35]:
filtered_bikes_rebalanced = 0
final_rebalancing_dict = {}
for k, v in filtered_rebalancing.items():
    if filtered_bikes_rebalanced < max_bikes_rebalanced:
        final_rebalancing_dict[k] = v
        filtered_bikes_rebalanced += v

In [36]:
rebalancing_df = pd.DataFrame(final_rebalancing_dict.items(), columns = ['dock_ids', 'num_bikes'])

In [37]:
rebalancing_df[['dock_id_receive', 'dock_id_give']] = rebalancing_df['dock_ids'].tolist()
rebalancing_df.drop(['dock_ids'], axis = 1, inplace = True)

In [38]:
data_df = query[['dock_id', 'latitude', 'longitude']]

In [39]:
new_docks = new[['dock_id', 'dock_name']]

In [40]:
rebalancing_df = rebalancing_df.merge(data_df, how = 'left', left_on = 'dock_id_receive', right_on = 'dock_id').rename(
    columns = {'latitude': 'latitude_receive', 'longitude': 'longitude_receive'})
rebalancing_df = rebalancing_df.merge(data_df, how = 'left', left_on = 'dock_id_give', right_on = 'dock_id').rename(
    columns = {'latitude': 'latitude_give', 'longitude': 'longitude_give'})
rebalancing_df.drop(['dock_id_x', 'dock_id_y'], axis = 1, inplace = True)

In [41]:
rebalancing_df = rebalancing_df.merge(new_docks, how = 'left', left_on = 'dock_id_receive', right_on = 'dock_id').rename(
    columns = {'dock_name': 'dock_name_receive'})
rebalancing_df = rebalancing_df.merge(new_docks, how = 'left', left_on = 'dock_id_give', right_on = 'dock_id').rename(
    columns = {'dock_name': 'dock_name_give'})
rebalancing_df.drop(['dock_id_x', 'dock_id_y'], axis = 1, inplace = True)

In [42]:
GREEN_RGB = [0, 255, 0, 150]
RED_RGB = [240, 100, 0, 150]

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data = rebalancing_df,
    get_width="num_bikes",
    get_source_position=["longitude_give", "latitude_give"],
    get_target_position=["longitude_receive", "latitude_receive"],
    get_tilt=15,
    get_source_color=GREEN_RGB,
    get_target_color=RED_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=40.74, longitude=-74, bearing=290, pitch=50, zoom=12)


TOOLTIP_TEXT = {"html": "{num_bikes} bikes need rebalancing from<br />{dock_name_give} to {dock_name_receive}"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT, map_style = 'light')
r.to_html("monday_12pm_rebalancing.html")
with open('monday_12pm_rebalancing.json', 'w') as outfile:
    json.dump(r.to_json(), outfile)

In [43]:
bikes_rebalanced

198

In [44]:
filtered_bikes_rebalanced

147

In [45]:
data_low['deficit'].sum()

645

In [46]:
data_high['surplus'].sum()

432

In [47]:
data_low.head(5)

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,datetime,avail_bikes_proportion,deficit
79,12,1,14,40.763441,-73.982681,57,2.85,0,0,1,2.614802,469,2021-12-19 14:00:00,0.045874,16
7,12,1,14,40.748901,-73.976049,45,2.85,0,0,1,2.609528,167,2021-12-19 14:00:00,0.057990,12
6,12,1,14,40.753231,-73.970325,47,2.85,0,5,1,3.816069,164,2021-12-19 14:00:00,0.081193,12
145,12,1,14,40.775730,-73.956753,47,2.85,0,1,2,3.870391,3146,2021-12-19 14:00:00,0.082349,12
144,12,1,14,40.765005,-73.958185,59,2.85,0,5,1,7.452996,3141,2021-12-19 14:00:00,0.126322,12


In [48]:
data_high.head(5)

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,datetime,avail_bikes_proportion,surplus
305,12,1,14,40.742754,-74.007474,39,2.85,0,5,3,36.828159,3472,2021-12-19 14:00:00,0.944312,11
284,12,1,14,40.724305,-73.996010,40,2.85,0,5,3,37.205040,3427,2021-12-19 14:00:00,0.930126,11
285,12,1,14,40.685068,-73.977908,39,2.85,0,2,3,36.585306,3429,2021-12-19 14:00:00,0.938085,11
48,12,1,14,40.716021,-73.999744,43,2.85,0,2,3,38.709895,355,2021-12-19 14:00:00,0.900230,10
303,12,1,14,40.724947,-74.001659,42,2.85,0,2,0,38.082064,3467,2021-12-19 14:00:00,0.906716,10


In [49]:
table = rebalancing_df[['dock_name_give', 'dock_name_receive', 'num_bikes']]
table = table.rename(columns = {'dock_name_give': 'dock origin', 'dock_name_receive': 'dock destination', 'num_bikes': 'number of bikes'})

table_html = table.to_html()

text_file = open("rebalancing_table.html", "w")
text_file.write(table_html)
text_file.close()